![](https://media.giphy.com/media/64aBXTVfd90zyUH2da/giphy.gif)


In [ ]:
# Ignore  the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

from random import shuffle
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.callbacks import ReduceLROnPlateau

# specifically for cnn
from keras.layers import Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
 
import tensorflow as tf
import random as rn

# specifically for manipulating zipped images and getting numpy arrays of pixel values of images.
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

#TL pecific modules
from keras.applications.vgg16 import VGG16

## Preparing the Data

In [ ]:
X=[]
Z=[]

X1 = []
Z1 = []
IMG_SIZE=224
NORMAL_TEST_DIR='../../../../mlwithpatu/chest_xray/test/NORMAL'
PNEUMONIA_TEST_DIR='../../../../mlwithpatu/chest_xray/test/PNEUMONIA'

NORMAL_TRAIN_DIR='../../../../mlwithpatu/chest_xray/train/NORMAL'
PNEUMONIA_TRAIN_DIR='../../../../mlwithpatu/chest_xray/train/PNEUMONIA'


In [ ]:
def assign_label(img,disease_type):
    return disease_type

In [ ]:
def make_train_data(disease_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,disease_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))

        X.append(np.array(img))
        Z.append(str(label))

        
        

In [ ]:
def make_test_data(disease_type,DIR):
    for img in tqdm(os.listdir(DIR)):
        label=assign_label(img,disease_type)
        path = os.path.join(DIR,img)
        img = cv2.imread(path,cv2.IMREAD_COLOR)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))

        X1.append(np.array(img))
        Z1.append(str(label))

        

In [ ]:
make_test_data('Normal',NORMAL_TEST_DIR)
print(len(X1))

In [ ]:
make_test_data('Pneumonia',PNEUMONIA_TEST_DIR)
print(len(X1))

In [ ]:
make_train_data('Normal',NORMAL_TRAIN_DIR)
print(len(X))

In [ ]:
make_train_data('Pneumonia',PNEUMONIA_TRAIN_DIR)
print(len(X))

In [ ]:
C = list(zip(X, Z))

shuffle(C)

X, Z = zip(*C)

C1 = list(zip(X1, Z1))

shuffle(C1)

X1, Z1 = zip(*C1)


## Visualizing some Random Images on the Train List

In [ ]:
fig,ax=plt.subplots(5,2)
fig.set_size_inches(15,15)
for i in range(5):
    for j in range (2):
        l=rn.randint(0,len(Z))
        ax[i,j].imshow(X[l])
        ax[i,j].set_title('Type: '+Z[l])
        
plt.tight_layout()

## Label Encoding the Y array & then One Hot Encoding 

In [ ]:
le=LabelEncoder()
Y=le.fit_transform(Z)
Y=to_categorical(Y,2)
X=np.array(X)
X=X/255

In [ ]:
le=LabelEncoder()
Y1=le.fit_transform(Z1)
Y1=to_categorical(Y1,2)
X1=np.array(X1)
X1=X1/255

##  Giving better names for the arrays. (This part was for splitting the data before. But we split the data in the dataset level as best practice)

In [ ]:
x_train = X
y_train = Y
x_test = X1
y_test = Y1


## Setting the random seeds

In [ ]:
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)



## Specifying the Base Model

We used VGG16 mode as the base mode. With the input shape of 64,64,3

In [ ]:
base_model=VGG16(include_top=False, weights=None,input_shape=(224,224,3), pooling='avg')


In [ ]:
base_model.summary()

Adding our model, making it trainable and adding last layers.

In [ ]:
model=Sequential()
model.add(base_model)

model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dense(2,activation='softmax'))

for layer in base_model.layers:
    layer.trainable=True


Using datagen to add random rotations, zooms, shifts and flips to the train dataset

In [ ]:
datagen = ImageDataGenerator() 


datagen.fit(x_train)

In [ ]:
epochs=3
batch_size=128
red_lr=ReduceLROnPlateau(monitor='val_acc', factor=0.1, min_delta=0.0001, patience=2, verbose=1)

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer=Adam(lr=1e-4),loss='categorical_crossentropy',metrics=['accuracy'])


In [ ]:
History = model.fit_generator(datagen.flow(x_train,y_train, batch_size=batch_size, shuffle=True),
                              epochs = epochs, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=x_train.shape[0] // batch_size)

Training the model. In the plots under, we can see how the accuracy and loss values changed over epochs.

In [ ]:
plt.plot(History.history['accuracy'])
plt.plot(History.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
plt.plot(History.history['loss'])
plt.plot(History.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['train', 'test'])
plt.show()

In [ ]:
# getting predictions on val set.
pred=model.predict(x_test)
pred_digits=np.argmax(pred,axis=1)

In [ ]:
# now storing some properly as well as misclassified indexes'.
i=0
prop_class=[]
mis_class=[]

for i in range(len(y_test)):
    if(np.argmax(y_test[i][0])==pred_digits[i]):
        prop_class.append(i)
    if(len(prop_class)==8):
        break

i=0
for i in range(len(y_test)):
    if(not np.argmax(y_test[i])==pred_digits[i]):
        mis_class.append(i)
    if(len(mis_class)==8):
        break

In [ ]:
warnings.filterwarnings('ignore')

count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[prop_class[count]])
        ax[i,j].set_title("Predicted Diagnosis : "+str(le.inverse_transform([pred_digits[prop_class[count]]]))+"\n"+"Actual Diagnosis : "+str(le.inverse_transform([np.argmax([y_test[prop_class[count]]])])))
        plt.tight_layout()
        count+=1

In [ ]:
warnings.filterwarnings('always')

count=0
fig,ax=plt.subplots(4,2)
fig.set_size_inches(15,15)
for i in range (4):
    for j in range (2):
        ax[i,j].imshow(x_test[mis_class[count]])
        ax[i,j].set_title("Predicted Diagnosis : "+str(le.inverse_transform([pred_digits[mis_class[count]]]))+"\n"+"Actual Diagnosis : "+str(le.inverse_transform([np.argmax([y_test[mis_class[count]]])])))
        plt.tight_layout()
        count+=1